In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import sys
from pathlib import Path

root_path = Path("./drive/Othercomputers/HomePC/")
project_path = root_path / "Segmentation-Pipeline"
if project_path not in sys.path:
  sys.path.append(project_path)

!pip install -r ./drive/Othercomputers/HomePC/Segmentation-Pipeline/requirements.txt

In [5]:
import torch

from research.diffusion_hippocampus.configs import main_config
from segmentation_pipeline import prepare_dataset_files, SegmentationTrainer, WandbLogger

dataset_path = root_path / "Drive_Datasets" / "Diffusion_MRI_cropped.tar"
dataset_path = prepare_dataset_files(dataset_path, ".")
context = main_config.get_context(
    device=torch.device("cuda"),
    variables={"DATASET_PATH": str(root_path / "Diffusion_MRI_cropped")},
    fold=0,
    predict_hbt=False
)
context.init_components()

trainer: SegmentationTrainer = context.trainer
trainer.train(
    context=context,
    max_iterations=100000,
    preload_training_data=False,
    preload_validation_data=False,
    num_workers=0,
    validation_batch_size=16,
    logger=WandbLogger("dmri-hippo-seg-debugging", root_path / "Collab-Checkpoints")
)

ModuleNotFoundError: ignored